# 데이터 로드

In [115]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
def load_dataset(csv_path, TRAIN_RATIO=0.8):
    
    global X, y, X_train, X_test, y_train, y_test, df
    
    # 데이터셋 로드
    df = pd.read_csv(csv_path)
    
    # 성별 원핫 인코딩
    df=pd.get_dummies(df,columns=['Sex'])
    
    # 학습 데이터 분리
    X = df.drop('Rings', axis=1)
    y = df['Rings']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_RATIO, random_state = 83)
    
    return X_train, X_test, y_train, y_test

csv_path = '../colabo/Data/Regression_data.csv'
X_train, X_test, y_train, y_test = load_dataset(csv_path)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3341, 10) (836, 10) (3341,) (836,)


In [116]:
# 전복 전체 무게가 살 + 내장 + 껍질보다 적게 나가는 경우는 말이 안됨
body = X_train['Whole weight'] - (X_train['Shucked weight'] + X_train['Viscera weight'] + X_train['Shell weight'])
X_train['body'] = body

index = X_train[X_train['body'] < 0].index

body = X_test['Whole weight'] - (X_test['Shucked weight'] + X_test['Viscera weight'] + X_test['Shell weight'])
X_test['body'] = body

index2 = X_test[X_test['body'] < 0].index

print(index.shape, index2.shape)

(118,) (37,)


In [117]:
X_train.drop(index, axis=0, inplace=True)
X_test.drop(index2, axis=0, inplace=True)
y_train.drop(index, axis=0, inplace=True)
y_test.drop(index2, axis=0, inplace=True)

In [118]:
"""
#2. 샘플 노이즈 추가 (기존 데이터셋에 완전히 랜덤한 노이즈 샘플을 추가하는 방법)
n_samples = len(df)
n_features = len(df.columns)

# 연속형 특성
continuous_columns = ['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
continuous_noise_samples = np.zeros((n_samples, len(continuous_columns))) # 노이즈 샘플 초기화
for i, column in enumerate(continuous_columns):
    min_val = df[column].min() # df[column]의 최소값
    max_val = df[column].max() # df[column]의 최대값
    noise = np.random.uniform(min_val, max_val, size=n_samples) # 최소값과 최대값 사이로 노이즈 값 생성
    continuous_noise_samples[:, i] = noise # 초기화 해놓은 샘플 배열에 노이즈 추가

# 범주형 특성
categorical_columns = ['Sex_F', 'Sex_M', 'Sex_I']
categorical_noise_samples = np.zeros((n_samples, len(categorical_columns)))
for i, column in enumerate(categorical_columns):
    # 원본 데이터 분포에 기반하여 랜덤 카테고리 생성
    probabilities = df[column].value_counts(normalize=True).values # 원본 데이터 분포 확인
    categories = np.random.choice([0, 1], size=n_samples, p=probabilities)
    categorical_noise_samples[:, i] = categories

# 연속형 특성과 범주형 특성 합침
noise_samples = np.hstack([continuous_noise_samples, categorical_noise_samples])

# 데이터 프레임으로 타입 변경 후 원본 데이터에 추가
noise_df = pd.DataFrame(noise_samples, columns=df.columns)
augmented_df = pd.concat([df, noise_df])

augmented_df.shape

# 학습 데이터 분리
X = augmented_df.drop('Rings', axis=1)
y = augmented_df['Rings']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 83)
"""

"\n#2. 샘플 노이즈 추가 (기존 데이터셋에 완전히 랜덤한 노이즈 샘플을 추가하는 방법)\nn_samples = len(df)\nn_features = len(df.columns)\n\n# 연속형 특성\ncontinuous_columns = ['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']\ncontinuous_noise_samples = np.zeros((n_samples, len(continuous_columns))) # 노이즈 샘플 초기화\nfor i, column in enumerate(continuous_columns):\n    min_val = df[column].min() # df[column]의 최소값\n    max_val = df[column].max() # df[column]의 최대값\n    noise = np.random.uniform(min_val, max_val, size=n_samples) # 최소값과 최대값 사이로 노이즈 값 생성\n    continuous_noise_samples[:, i] = noise # 초기화 해놓은 샘플 배열에 노이즈 추가\n\n# 범주형 특성\ncategorical_columns = ['Sex_F', 'Sex_M', 'Sex_I']\ncategorical_noise_samples = np.zeros((n_samples, len(categorical_columns)))\nfor i, column in enumerate(categorical_columns):\n    # 원본 데이터 분포에 기반하여 랜덤 카테고리 생성\n    probabilities = df[column].value_counts(normalize=True).values # 원본 데이터 분포 확인\n    categories = np.random.choice([0, 1], size=n_sa

In [119]:
import numpy as np

# train 껍질의 넓이 ( a * b * π)
area = 0.5 * X_train['Length'] * 0.5 * X_train['Diameter'] * np.pi
X_train['Area'] = area

# test 껍질의 넓이 
area2 = 0.5 * X_test['Length'] * 0.5 * X_test['Diameter'] * np.pi
X_test['Area'] = area2

In [120]:
# train 껍질의 둘레 (근사) ( 2π*(0.5 * √(a^2 + b^2)))
perimeter = np.pi * np.sqrt(0.5 * ((X_train['Length'] ** 2) + (X_train['Diameter'] ** 2)))
X_train['Perimeter'] = perimeter

# test 껍질의 둘레 (근사) ( 2π*(0.5 * √(a^2 + b^2)))
perimeter2 = np.pi * np.sqrt(0.5 * ((X_test['Length'] ** 2) + (X_test['Diameter'] ** 2)))
X_test['Perimeter'] = perimeter2

X_train.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Sex_F,Sex_I,Sex_M,body,Area,Perimeter
461,0.585,0.465,0.170,0.9915,0.3865,0.2240,0.265,1,0,0,0.1160,0.213648,1.660072
2835,0.570,0.420,0.140,0.8745,0.4160,0.1650,0.250,0,0,1,0.0435,0.188024,1.572837
1378,0.620,0.500,0.150,1.2930,0.5960,0.3135,0.354,1,0,0,0.0295,0.243473,1.769361
2569,0.460,0.345,0.115,0.4215,0.1895,0.1020,0.111,0,1,0,0.0190,0.124643,1.277329
369,0.690,0.560,0.215,1.7190,0.6800,0.2990,0.470,1,0,0,0.2700,0.303478,1.974085


In [121]:
# 타겟값과 각 변수들 간의 상관관계
co = X_train.corrwith(y_train)

# 상관계수를 내림차순으로 정리
print(co.sort_values(ascending=False))

# 절대값
co_abs = abs(co)

Shell weight      0.619796
Diameter          0.559512
Perimeter         0.548978
Length            0.539317
Height            0.534439
body              0.533203
Area              0.532949
Whole weight      0.525487
Viscera weight    0.489945
Shucked weight    0.403014
Sex_F             0.242466
Sex_M             0.178722
Sex_I            -0.431255
dtype: float64


In [122]:
columns = ['Shell weight', 'Diameter', 'Perimeter', 'Length', 'Height', 'Area', 'Viscera weight', 'Shucked weight', 'Whole weight', 'Sex_F', 'Sex_M', 'Sex_I']
X_train_2 = X_train[columns]
X_test_2 = X_test[columns]
X_train_2.head()

,Shell weight,Diameter,Perimeter,Length,Height,Area,Viscera weight,Shucked weight,Whole weight,Sex_F,Sex_M,Sex_I
461,0.265,0.465,1.660072,0.585,0.170,0.213648,0.2240,0.3865,0.9915,1,0,0
2835,0.250,0.420,1.572837,0.570,0.140,0.188024,0.1650,0.4160,0.8745,0,1,0
1378,0.354,0.500,1.769361,0.620,0.150,0.243473,0.3135,0.5960,1.2930,1,0,0
2569,0.111,0.345,1.277329,0.460,0.115,0.124643,0.1020,0.1895,0.4215,0,0,1
369,0.470,0.560,1.974085,0.690,0.215,0.303478,0.2990,0.6800,1.7190,1,0,0


In [123]:
from sklearn.preprocessing import MinMaxScaler

# 스케일링할 피처 선택
scaling_features = columns[:-3]  # 원핫인코딩되지 않은 연속형 또는 순서형 변수들

# 스케일링
scaler = MinMaxScaler()
X_train_scaled = X_train_2.copy()  # 원본 데이터 복사
X_test_scaled = X_test_2.copy()    # 원본 데이터 복사
X_train_scaled[scaling_features] = scaler.fit_transform(X_train_2[scaling_features])
X_test_scaled[scaling_features] = scaler.transform(X_test_2[scaling_features])

In [124]:
"""
# 노이즈 데이터 추가
# 1. 특성 노이즈 추가 (기존 데이터셋의 특성들에 랜덤 노이즈를 직접 추가하는 방법)
# 노이즈 팩터 설정
noise_factor = 0.001
numeric_columns = ['Shell weight', 'Diameter', 'Perimeter', 'Length', 'Height', 'Area', 'Viscera weight', 'Shucked weight', 'Whole weight'] # 수치형 데이터에만 적용해야 함

# feature에 노이즈 적용
X_train_noisy = X_train_scaled.copy()
X_train_noisy[numeric_columns] = X_train_noisy[numeric_columns] + noise_factor * np.random.randn(*X_train_noisy[numeric_columns].shape)

# 노이즈 값을 원본 데이터의 최솟값과 최댓값 사이로 제한
for col in numeric_columns:
    X_train_noisy[col] = X_train_noisy[col] + noise_factor * np.random.randn(*X_train_noisy[col].shape)
    X_train_noisy[col] = np.clip(X_train_noisy[col], X_train[col].min(), X_train[col].max())

X_train_noisy
"""

"\n# 노이즈 데이터 추가\n# 1. 특성 노이즈 추가 (기존 데이터셋의 특성들에 랜덤 노이즈를 직접 추가하는 방법)\n# 노이즈 팩터 설정\nnoise_factor = 0.001\nnumeric_columns = ['Shell weight', 'Diameter', 'Perimeter', 'Length', 'Height', 'Area', 'Viscera weight', 'Shucked weight', 'Whole weight'] # 수치형 데이터에만 적용해야 함\n\n# feature에 노이즈 적용\nX_train_noisy = X_train_scaled.copy()\nX_train_noisy[numeric_columns] = X_train_noisy[numeric_columns] + noise_factor * np.random.randn(*X_train_noisy[numeric_columns].shape)\n\n# 노이즈 값을 원본 데이터의 최솟값과 최댓값 사이로 제한\nfor col in numeric_columns:\n    X_train_noisy[col] = X_train_noisy[col] + noise_factor * np.random.randn(*X_train_noisy[col].shape)\n    X_train_noisy[col] = np.clip(X_train_noisy[col], X_train[col].min(), X_train[col].max())\n\nX_train_noisy\n"

In [125]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [126]:
# 사용자 정의 평가 지표 클래스
import tensorflow as tf

class EvalAccuracy(tf.keras.metrics.Metric): # TensorFlow의 Metric 클래스를 상속 받음

    def __init__(self, name="eval_accuracy", **kwargs): # 부모 클래스의 __init__() 메소드를 호출하여 필요한 초기화를 수행
        super(EvalAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight(name="ctp", initializer="zeros")
        # add_weight() 메소드를 사용하여 평가 지표를 계산하는데 필요한 변수를 생성(각 배치에서의 평가 결과를 누적하기 위해)
        # add_weight() 는 텐서플로우 Layer 클래스의 메서드(새로운 가중치를 추가하는 기능, 여기서는 평가 지표를 계산하는 데 사용되는 일종의 내부 변수를 의미)
        # 이 구문이 실행되면, EvalAccuracy 인스턴스는 새로운 가중치를 추가하고 그 가중치를 self.correct에 저장한다.
        # 이 self.correct는 update_state() 메서드에서 업데이트되며, '현재까지 처리한 모든 배치에 대한 평가 지표의 평균을 저장'한다.

    def update_state(self, y_true, y_predict, sample_weight=None):
        value = tf.abs((y_predict - y_true) / y_true)
        self.correct.assign(tf.reduce_mean(value)) # 오차율을 계산해서 correct 변수에 누적한 후, assign() 메소드를 사용하여 correct 변수의 값을 업데이트

    def result(self):
        return 1 - self.correct

    def reset_states(self):
        # 에포크마다 평가 지표 초기화
        self.correct.assign(0.)

In [172]:
# 베이스모델
import numpy as np

def Base_Model(LEARNING_RATE=0.01):
    
    np.random.seed(42)
    tf.random.set_seed(42)
    
    global X, y, X_train, X_test, y_train, y_test, df
    
    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')
    
    
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=64, activation='relu', input_shape=(len(X_train_2.keys()),)),
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dense(units=256, activation='relu'),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dense(units=256, activation='relu'),
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1)
    ])
    """
    
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=64, input_shape=(len(X_train_2.keys()),)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Dense(units=128),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Dense(units=256),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Dense(units=512),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Dense(units=256),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Dense(units=128),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Dense(units=64),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Dense(units=1)
    ])
    """
    
    # 옵티마이저와 손실 함수 설정
    optimizer = tf.keras.optimizers.Adam(
                                        learning_rate=0.01,
                                        beta_1=0.9,
                                        beta_2=0.99,
                                        epsilon=1e-08
                                        )

    model.compile(loss='mean_squared_error',
                  optimizer=optimizer,
                  metrics=[EvalAccuracy()])
    
    return model

model = Base_Model()
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_112 (Dense)           (None, 64)                832       
                                                                 
 dense_113 (Dense)           (None, 128)               8320      
                                                                 
 dense_114 (Dense)           (None, 256)               33024     
                                                                 
 dense_115 (Dense)           (None, 512)               131584    
                                                                 
 dense_116 (Dense)           (None, 256)               131328    
                                                                 
 dense_117 (Dense)           (None, 128)               32896     
                                                                 
 dense_118 (Dense)           (None, 64)              

In [158]:
# 학습 자동 중단 설정
es = EarlyStopping(monitor='loss', patience=50, mode='min')
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=40, mode='min')

In [181]:
# 하이퍼파라미터 세팅
LEARNING_RATE = 0.01
EPOCHS = 200
MB_SIZE = 500 # (5000)
REPORT = 1
TRAIN_RATIO = 0.8

np.random.seed(42)
tf.random.set_seed(42)

# 모델 학습
history = model.fit(
  X_train_scaled, y_train,
  batch_size=MB_SIZE,
  validation_split = 0.2,
  verbose=1,
  epochs=EPOCHS,
  callbacks=[es, rlrp]
  )

Epoch 1/200


c:\Users\k10dh\anaconda3\envs\TeamProject\lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric EvalAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


6/6 [==============================] - 0s 33ms/step - loss: 3.1530 - eval_accuracy: 0.8718 - val_loss: 4.7379 - val_eval_accuracy: 0.8401 - lr: 4.0000e-04
Epoch 2/200
6/6 [==============================] - 0s 17ms/step - loss: 3.1573 - eval_accuracy: 0.8736 - val_loss: 4.7426 - val_eval_accuracy: 0.8372 - lr: 4.0000e-04
Epoch 3/200
6/6 [==============================] - ETA: 0s - loss: 3.1633 - eval_accuracy: 0.8605

c:\Users\k10dh\anaconda3\envs\TeamProject\lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric EvalAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


6/6 [==============================] - 0s 17ms/step - loss: 3.1633 - eval_accuracy: 0.8605 - val_loss: 4.7998 - val_eval_accuracy: 0.8354 - lr: 4.0000e-04
Epoch 4/200
6/6 [==============================] - 0s 19ms/step - loss: 3.1498 - eval_accuracy: 0.8775 - val_loss: 4.7419 - val_eval_accuracy: 0.8396 - lr: 4.0000e-04
Epoch 5/200
6/6 [==============================] - 0s 17ms/step - loss: 3.1669 - eval_accuracy: 0.8609 - val_loss: 4.7301 - val_eval_accuracy: 0.8400 - lr: 4.0000e-04
Epoch 6/200
6/6 [==============================] - 0s 18ms/step - loss: 3.1338 - eval_accuracy: 0.8694 - val_loss: 4.7493 - val_eval_accuracy: 0.8409 - lr: 4.0000e-04
Epoch 7/200
6/6 [==============================] - 0s 16ms/step - loss: 3.1565 - eval_accuracy: 0.8756 - val_loss: 4.7070 - val_eval_accuracy: 0.8413 - lr: 4.0000e-04
Epoch 8/200
6/6 [==============================] - 0s 16ms/step - loss: 3.1554 - eval_accuracy: 0.8877 - val_loss: 4.7640 - val_eval_accuracy: 0.8380 - lr: 4.0000e-04
Epoch 9/20

In [182]:
import matplotlib.pyplot as plt
"""
# 손실 그래프
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 훈련 및 검증 평가 지표 추출
train_accuracy = history.history['eval_accuracy']
val_accuracy = history.history['val_eval_accuracy']

# 평가 지표 그래프 그리기
plt.plot(train_accuracy, label='Train Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
"""
# 테스트 세트 평가
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", round(loss, 3))
print("Test Accuracy:", round(accuracy, 3))


25/25 [==============================] - 0s 2ms/step - loss: 5.2419 - eval_accuracy: 0.8736
Test Loss: 5.242
Test Accuracy: 0.874


c:\Users\k10dh\anaconda3\envs\TeamProject\lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric EvalAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


In [17]:
# 모델 가중치 및 편향 저장 (save_weights 메서드를 사용하면 가중치와 편향 모두를 저장)
model.save_weights('model_weights.h5')

# 다음과 같이 저장한 가중치 호출 가능
new_model = Base_Model()
new_model.load_weights('model_weights.h5')

In [18]:
# 테스트 세트 평가
loss, accuracy = new_model.evaluate(X_test_scaled, y_test)
print("Test Loss:", round(loss, 3))
print("Test Accuracy:", round(accuracy, 3))

25/25 [==============================] - 0s 3ms/step - loss: 4.3920 - eval_accuracy: 0.8879
Test Loss: 4.392
Test Accuracy: 0.888
